In [1]:
import glob
import os
import shutil

import numpy as np
import xarray as xr
import rioxarray

In [2]:
maindownloadfolder=r'D:\Data\BARRAv2\downloadedwind'
subsetfolder=r'D:\Data\BARRAv2'

Lost the wind code. download and read in uwind and vwind. after fixing projection, use rioxarray reproject_match to line up the grids. then clip, then merge. Merge didn't work with two variables for some reason, so easier to clip both and then merge, rather than merge and then clip.

In [3]:
stream='slv'
variables=['av_uwnd10m','av_vwnd10m']
#os.mkdir(subsetfolder + "\\Coorong\\")
#os.mkdir(subsetfolder + "\\CLLMR\\")

In [4]:
for year in range(1990,2010):
        for variable in variables:
        
            downloadfolder=maindownloadfolder + "\\" + variable + "\\" + str(year)

            if not os.path.exists(maindownloadfolder + "\\" + variable):
                os.mkdir(maindownloadfolder + "\\" + variable)

            if not os.path.exists(downloadfolder):
                os.mkdir(downloadfolder)
            
        for month in range(1,13):
            
            #download both uwing and vwind for this month.
            for variable in variables:
                
                downloadfolder=maindownloadfolder + "\\" + variable + "\\" + str(year)
            
                if not os.path.exists(downloadfolder + "\\" + str(month)):
                    os.mkdir(downloadfolder + "\\" + str(month))

                raijinfolder='/g/data/ma05/BARRA_AD/v1/forecast/'+stream+'/'+variable+'/'+str(year)+'/'+str(month).zfill(2)+'/*'
                os.system('C:/devapps/PuTTYPortable/pscp.exe -pw Coldbluewater1 mg3808@gadi.nci.org.au:'+ raijinfolder + " " + downloadfolder + "\\" + str(month))

            #load in and project to match mesh
            downloadfolder=maindownloadfolder + "\\" + variables[0] + "\\" + str(year)
            data = xr.open_mfdataset(glob.glob(downloadfolder + "\\" + str(month) + '\\*.nc'),combine='by_coords')
            data=data[[variables[0]]]
            data.rio.write_crs("+proj=longlat +R=6371229 +no_defs +type=crs",inplace=True)
            uWind=data.rio.reproject("EPSG:28354")

            downloadfolder=maindownloadfolder + "\\" + variables[1] + "\\" + str(year)
            data = xr.open_mfdataset(glob.glob(downloadfolder + "\\" + str(month) + '\\*.nc'),combine='by_coords')
            data=data[[variables[1]]]
            data.rio.write_crs("+proj=longlat +R=6371229 +no_defs +type=crs",inplace=True)
            vWind=data.rio.reproject("EPSG:28354").rio.reproject_match(uWind)
       
    
            filename= str(year)+str(month).zfill(2)+'.nc'
        
            #clip, merge and output. Merge before clip didn't work.
            CoorongU=uWind.rio.clip_box(298039,5985864,384367,6069563)
            CoorongV=vWind.rio.clip_box(298039,5985864,384367,6069563)
            CoorongU=CoorongU.merge(CoorongV)
            CoorongU.to_netcdf(subsetfolder + "\\Coorong\\av_wnd10\\" + filename)
            CoorongU.close()
            CoorongV.close()

            CLLMRU=uWind.rio.clip_box(296887,5985833,384607,6199011)
            CLLMRV=vWind.rio.clip_box(296887,5985833,384607,6199011)
            CLLMRU=CLLMRU.merge(CLLMRV)
            CLLMRU.to_netcdf(subsetfolder + "\\CLLMR\\av_wnd10\\" + filename)
            CLLMRU.close()
            CLLMRV.close()

            data.close()
            uWind.close()
            vWind.close()
            
        #at the end of the year, clear out the downloaded data
        for variable in variables:
        
            downloadfolder=maindownloadfolder + "\\" + variable + "\\" + str(year)
            
            try:
                shutil.rmtree(downloadfolder)
            except:
                print("")